In [1]:
cd ..

/home/jevans96/umd_icecube_analysis_tutorial


In [2]:
__author__ = 'John Evans'
__copyright__ = ''
__credits__ = ['John Evans', 'Jason Fan', 'Michael Larson']
__license__ = 'Apache License 2.0'
__version__ = '0.0.1'
__maintainer__ = 'John Evans'
__email__ = 'jevans96@umd.edu'
__status__ = 'Development'

"""
Docstring
"""

import numpy as np
from i3pubtools import tools
from i3pubtools import time_profiles
from i3pubtools.llhs import ps_flare_llh
# from test import test_data
import glob
from scipy import stats
from matplotlib import pyplot as plt
import cProfile

np.random.seed(0)

In [3]:
# Load in both the simulation and the data

# # Where is the dataset stored?
dataset = '/data/mjlarson/data/'

# Read in all of the data files
data_files = dataset + '/IC86_*exp.npy'
data = tools.read(glob.glob(data_files))

# Read in all of the MC files 
sim_files = dataset + '/IC86*MC*npy'
sim = tools.read(glob.glob(sim_files))

# trim data
data = data[:1000]

# data_size = 10000
# sim_size = 100000

# data = test_data.get_random_data(data_size)
# sim = test_data.get_random_sim(sim_size)

# Set the angular error floor to 0.2 degrees
data['angErr'][data['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
sim['angErr'][sim['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)


# Show the possible keys available here:
print('Events available in simulation: {}'.format(len(sim)))
print('Keys available in simulation:')
print(sorted(sim.dtype.names))
print()

print('Events available in data: {}'.format(len(data)))
print('Keys available in data:')
print(sorted(data.dtype.names))

Events available in simulation: 11595414
Keys available in simulation:
['angErr', 'azi', 'dec', 'event', 'logE', 'ow', 'ra', 'run', 'subevent', 'time', 'trueDec', 'trueE', 'trueRa', 'zen']

Events available in data: 1000
Keys available in data:
['angErr', 'azi', 'dec', 'event', 'logE', 'ra', 'run', 'subevent', 'time', 'zen']


In [4]:
# # Also load in the 'GoodRunList' (GRL), a file that tells
# # us when the detector was taking good data. 
grl_files = dataset + '/GRL/IC86_*exp.npy'
grl = tools.read(glob.glob(grl_files))
# grl = test_data.get_random_grl(data)


# Show the keys available in the GRL
print('Keys available in the GoodRunList:')
print(sorted(grl.dtype.names))
print(np.min(grl['start']))
print(np.max(grl['stop']))
gammas = np.arange(-4.25, -0.5, 0.25)
bins   = np.array([np.linspace(-1, 1, 51), # sin(dec)
                   np.linspace(1, 8, 51)]) # logE

Keys available in the GoodRunList:
['events', 'livetime', 'run', 'start', 'stop']
55694.99047453704
58309.08234953694


In [5]:
# Try this with a gaussian signal + uniform background.
outfile = './test/test_outputs/ps_flare_llh-test.npy'
infile = './test/test_outputs/ps_flare_llh-test.npy'

args = (data, sim, grl, gammas, bins)

kwargs = {
    'source': {'ra':np.pi/2, 'dec':np.pi/6},
    'signal_time_profile': time_profiles.GaussProfile(56102.5, 0.01), 
    'background_time_profile': time_profiles.UniformProfile(56102.48, 56102.52),
}

In [6]:
# Run this block once

#cProfile.run('ps_flare_llh.PsFlareLLH(*args, outfile = outfile, **kwargs)')
llh_object = ps_flare_llh.PsFlareLLH(*args, outfile = outfile, **kwargs)

  0%|          | 0/15 [00:00<?, ?it/s]/home/jevans96/umd_icecube_analysis_tutorial/i3pubtools/llhs/ps_flare_llh.py:128: RuntimeWarning: divide by zero encountered in true_divide
  ratio = sig_h / bg_h
/home/jevans96/umd_icecube_analysis_tutorial/i3pubtools/llhs/ps_flare_llh.py:128: RuntimeWarning: invalid value encountered in true_divide
  ratio = sig_h / bg_h
/home/jevans96/umd_icecube_analysis_tutorial/i3pubtools/llhs/ps_flare_llh.py:134: RuntimeWarning: invalid value encountered in greater
  good = np.isfinite(values) & (values>0)
100%|██████████| 15/15 [01:08<00:00,  4.58s/it]


In [7]:
# Afterwards, run this one to save time
llh_object = ps_flare_llh.PsFlareLLH(*args, infile = infile, **kwargs)

In [8]:
thousand_seconds = (1000.0)/(24*3600.)
llh_object.background_time_profile = time_profiles.UniformProfile(56044.0912963, 56044.0912963+thousand_seconds)
llh_object.signal_time_profile = llh_object.background_time_profile

# Produce a single trial. Here, we pass a 'seed'
# value to the function so that we can reproduce
# the trial later if we want to. If you want it to
# be random, set seed=None.
seed = 0
flux_norm = 1e-10
#cProfile.run('llh_object.produce_trial(flux_norm=flux_norm, random_seed=seed)')
#%timeit llh_object.produce_trial(flux_norm=flux_norm, random_seed=seed)
trial = llh_object.produce_trial(flux_norm=flux_norm, random_seed=seed, verbose=True)
print('Got {} events for this trial'.format(len(trial)))

print('Times:')
print(trial)

number of background events: 7429
number of signal events: 65
Got 7494 events for this trial
Times:
[(120157, 61672949, 0, 2.40144145,  0.88088172, 3.56915858, 2.45125344, 56044.09129653, 3.12386818, 0.04109117)
 (120160, 27485236, 0, 4.79832993,  0.63183663, 0.40303192, 2.20204638, 56044.09129745, 3.09296079, 0.0090418 )
 (120030, 49317620, 0, 1.52999572,  0.86583423, 0.23929013, 2.43761692, 56044.09130193, 3.21394864, 0.02522637)
 ...
 (120159, 29633456, 0, 5.46655076, -0.95248871, 1.34707644, 0.618721  , 56044.1028685 , 5.69406458, 0.00768739)
 (120159, 71272093, 0, 5.34703583, -0.91099765, 1.20152134, 0.65873879, 56044.10286919, 4.88118446, 0.00349066)
 (120159, 22327869, 0, 1.92805355, -0.96269831, 2.05720157, 0.60924264, 56044.10287004, 5.36902592, 0.00349066)]


In [9]:
# Run the function with our test trial. Make sure
# you understand what's getting returned!
ns = 50
gamma = -2
#%timeit llh_object.evaluate_ts(trial, ns=ns, gamma=gamma)
#cProfile.run('llh_object.evaluate_ts(trial, ns=ns, gamma=gamma)')
bestfit = llh_object.evaluate_ts(trial, ns=ns, gamma=gamma)
print(bestfit)

{'ts': nan, 'ns': 50.0, 'gamma': -2.0, 'uniform_tp_start': 56044.0912965318, 'uniform_tp_end': 56044.1028700378}


In [ ]:
# Run trials for a few different values of N. These settings should
# exactly mimic your previous script
thousand_seconds = (1000.0)/(24*3600.)
llh_object.background_time_profile = time_profiles.UniformProfile(56044.0912963, 56044.0912963+thousand_seconds)
llh_object.signal_time_profile = llh_object.background_time_profile

background_fits = llh_object.produce_n_trials(1000, test_ns=0.1)
signal_16_fits = llh_object.produce_n_trials(1000, flux_norm=1e-11, test_ns=0.1)
signal_15_fits = llh_object.produce_n_trials(1000, flux_norm=1e-10, test_ns=0.1)
signal_14_fits = llh_object.produce_n_trials(1000, flux_norm=1e-9, test_ns=0.1)

In [ ]:
# Plot the signal and background LLHs
fig, ax = plt.subplots(figsize=(10,6))
bins = np.linspace(0, 500, 50)

ax.hist(background_fits['ts'],
        bins = bins,
        weights = np.ones(background_fits.shape[0])/len(background_fits),
        histtype = 'step',
        color = 'k',
        linewidth = 3,
        alpha=0.5,
        label = r'N = 0')

ax.hist(signal_16_fits['ts'],
        bins = bins,
        weights = np.ones(signal_16_fits.shape[0])/len(signal_16_fits),
        histtype = 'step',
        linewidth = 3,
        alpha = 0.7,
        label = r'N = $10^{-16}$')

ax.hist(signal_15_fits['ts'],
        bins = bins,
        weights = np.ones(signal_15_fits.shape[0])/len(signal_15_fits),
        histtype = 'step',
        linewidth = 3,
        alpha = 0.7,
        label = r'N = $10^{-15}$')

ax.hist(signal_14_fits['ts'],
        bins = bins,
        weights = np.ones(signal_14_fits.shape[0])/len(signal_14_fits),
        histtype = 'step',
        linewidth = 3,
        alpha = 0.7,
        label = r'N = $10^{-14}$')

ax.legend(loc='upper right', fontsize=16,)

ax.grid(alpha=0.25)
ax.set_yscale('log')
ax.set_xlabel('TS', fontsize=16)
ax.set_ylabel('Fraction of Trials', fontsize=16)
ax.tick_params(which='both', labelsize=16)

In [ ]:
# Try it again, but using a gaussian signal this time.
# WARNING: The gaussian time profile is kind of hacked
# together and may not give the most reliable results.
# Use with caution!
thousand_seconds = (1000.0)/(24*3600.)
llh_object.background_time_profile = time_profiles.UniformProfile(56044.0912963, 56044.0912963+thousand_seconds)
llh_object.signal_time_profile = time_profiles.GaussProfile(56044.0912963 + thousand_seconds/2, thousand_seconds/10)

background_fits = llh_object.produce_n_trials(1000, test_ns=0.1)
signal_16_fits = llh_object.produce_n_trials(1000, flux_norm=1e-16, test_ns=0.1)
signal_15_fits = llh_object.produce_n_trials(1000, flux_norm=1e-15, test_ns=0.1)
signal_14_fits = llh_object.produce_n_trials(1000, flux_norm=1e-14, test_ns=0.1)

In [ ]:
# Plot the signal and background LLHs
fig, ax = plt.subplots(figsize=(10,6))
bins = np.linspace(0, 500, 50)

ax.hist(background_fits['ts'],
        bins = bins,
        weights = np.ones(background_fits.shape[0])/len(background_fits),
        histtype = 'step',
        color = 'k',
        linewidth = 3,
        alpha=0.5,
        label = r'N = 0')

ax.hist(signal_16_fits['ts'],
        bins = bins,
        weights = np.ones(signal_16_fits.shape[0])/len(signal_16_fits),
        histtype = 'step',
        linewidth = 3,
        alpha = 0.7,
        label = r'N = $10^{-16}$')

ax.hist(signal_15_fits['ts'],
        bins = bins,
        weights = np.ones(signal_15_fits.shape[0])/len(signal_15_fits),
        histtype = 'step',
        linewidth = 3,
        alpha = 0.7,
        label = r'N = $10^{-15}$')

ax.hist(signal_14_fits['ts'],
        bins = bins,
        weights = np.ones(signal_14_fits.shape[0])/len(signal_14_fits),
        histtype = 'step',
        linewidth = 3,
        alpha = 0.7,
        label = r'N = $10^{-14}$')

ax.legend(loc='upper right', fontsize=16,)

ax.grid(alpha=0.25)
ax.set_yscale('log')
ax.set_xlabel('TS', fontsize=16)
ax.set_ylabel('Fraction of Trials', fontsize=16)
ax.tick_params(which='both', labelsize=16)

In [ ]:
thousand_seconds = (50000.0)/(24*3600.)
# means  = [56044.0912963 + thousand_seconds/5,56044.0912963 + thousand_seconds/3,
#           56044.0912963 + thousand_seconds/4,56044.0912963 + thousand_seconds/5,
#           56044.0912963 + 3*thousand_seconds/4]
# sigmas = [thousand_seconds/10,thousand_seconds/11,
#           thousand_seconds/8,thousand_seconds/5,
#           thousand_seconds/20]
llh_object.background_time_profile = time_profiles.UniformProfile(56044.0912963, 56044.0912963+thousand_seconds)
# llh_object.signal_time_profile = n_gauss_profile(means, sigmas)
llh_object.signal_time_profile = time_profiles.GaussProfile(56044.0912963 + thousand_seconds/2, thousand_seconds/10)

trial = llh_object.produce_trial(flux_norm=1e-14)

In [ ]:
counts, bins, _ = plt.hist(trial['time'], bins = len(trial)//5)

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
X = trial['time'].reshape(-1, 1)
db = DBSCAN(eps=.01, min_samples=2).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

In [ ]:
labels = db.labels_
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]

In [ ]:
for k, col in zip(unique_labels, colors):
    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    
    plt.hist(xy, bins = bins)